In [1]:
!pip uninstall -y mediapipe
!pip install --upgrade pip
!pip install mediapipe==0.10.7 protobuf==3.20.3 absl-py==2.1.0 numpy==1.23.5 opencv-python==4.7.0.72 torch==2.0.1 torchvision==0.15.2 pillow==9.5.0 loguru==0.7.2

# Verify installation and test Mediapipe
import mediapipe as mp
try:
    mp.solutions.pose.Pose(static_image_mode=True)
    print(f"Mediapipe version: {mp.__version__} - Successfully initialized")
except Exception as e:
    print(f"Mediapipe initialization failed: {e}")

Found existing installation: mediapipe 0.10.11
Uninstalling mediapipe-0.10.11:
  Successfully uninstalled mediapipe-0.10.11
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.6/33.6 MB 112.4 MB/s eta 0:00:00
Mediapipe version: 0.10.7 - Successfully initialized


In [2]:
# Import modules
import cv2
import numpy as np
import mediapipe as mp
from loguru import logger
import torch
import torchvision.transforms as T
from PIL import Image
from google.colab import files
import logging
import os
import sys
import time

# Configure logging
logger.remove()
logger.add(logging.StreamHandler(), level="INFO", format="{time} {level} {message}")

class BodyMeasurementProcessor:
    def __init__(self, model_weights_path="dpt_large_384.pt"):
        # Initialize MediaPipe Pose
        try:
            self.mp_pose = mp.solutions.pose.Pose(
                static_image_mode=True,
                min_detection_confidence=0.6,
                min_tracking_confidence=0.6
            )
            self.mp_pose_low_conf = mp.solutions.pose.Pose(
                static_image_mode=True,
                min_detection_confidence=0.4,
                min_tracking_confidence=0.4
            )
        except Exception as e:
            logger.error(f"Failed to initialize MediaPipe Pose: {e}")
            raise
        # Initialize DPT model
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.depth_model = None
        try:
            self.depth_model = torch.hub.load("intel-isl/MiDaS", "DPT_Large", pretrained=True, force_reload=False)
            if os.path.exists(model_weights_path):
                state_dict = torch.load(model_weights_path, map_location=self.device)
                self.depth_model.load_state_dict(state_dict)
                logger.info("Loaded custom DPT model weights")
            self.depth_model.to(self.device)
            self.depth_model.eval()
        except Exception as e:
            logger.error(f"Failed to initialize DPT model: {e}")
            self.depth_model = None
        self.transform = T.Compose([
            T.Resize((384, 384)),
            T.ToTensor(),
            T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])

    def validate_image(self, image: np.ndarray) -> bool:
        """Validate if the image is suitable for processing."""
        if image is None or image.size == 0:
            logger.error("Invalid image: Empty or None")
            return False
        if image.shape[0] < 200 or image.shape[1] < 200:
            logger.error("Invalid image: Too small (minimum 200x200 pixels)")
            return False
        h, w = image.shape[:2]
        aspect_ratio = w / h
        if aspect_ratio < 0.5 or aspect_ratio > 2.0:
            logger.warning("Unusual aspect ratio. Ensure full-body capture with proper posture.")
        return True

    def detect_landmarks(self, image: np.ndarray) -> dict:
        """Detect pose landmarks using MediaPipe with fallback."""
        try:
            if not self.validate_image(image):
                return {}
            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            results = self.mp_pose.process(image_rgb)
            if not results.pose_landmarks:
                logger.warning("No landmarks detected with high confidence. Trying lower confidence...")
                results = self.mp_pose_low_conf.process(image_rgb)
                if not results.pose_landmarks:
                    logger.warning("No landmarks detected. Ensure clear, full-body pose with good lighting.")
                    return {}
            landmarks = {}
            for idx, landmark in enumerate(results.pose_landmarks.landmark):
                landmarks[mp.solutions.pose.PoseLandmark(idx).name.lower()] = {
                    "x": landmark.x * image.shape[1],
                    "y": landmark.y * image.shape[0],
                    "z": landmark.z * image.shape[1]
                }
            return landmarks
        except Exception as e:
            logger.error(f"Landmark detection failed: {e}")
            return {}

    def estimate_depth(self, image: np.ndarray) -> np.ndarray:
        """Estimate depth map using DPT, scaled to meters (0.15-0.4m range for body)."""
        if self.depth_model is None:
            logger.warning("DPT model not initialized, skipping depth estimation")
            return np.full(image.shape[:2], 0.25, dtype=np.float32)  # Default depth
        try:
            if not self.validate_image(image):
                return np.full(image.shape[:2], 0.25, dtype=np.float32)
            img = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
            img_tensor = self.transform(img).unsqueeze(0).to(self.device)
            with torch.no_grad():
                depth = self.depth_model(img_tensor)
            depth = torch.nn.functional.interpolate(
                depth.unsqueeze(1),
                size=image.shape[:2],
                mode="bicubic",
                align_corners=False
            ).squeeze().cpu().numpy()
            # Scale depth to 0.15-0.4m range
            depth = 0.15 + (0.25 * (depth - depth.min()) / (depth.max() - depth.min() + 1e-8))
            depth = np.clip(depth, 0.15, 0.4)
            return depth.astype(np.float32)
        except Exception as e:
            logger.error(f"Depth estimation failed: {e}")
            return np.full(image.shape[:2], 0.25, dtype=np.float32)

    def _get_average_depth_at_point(self, depth_map: np.ndarray, landmarks: dict, keys: list) -> float:
        """Get average depth at landmark points in meters."""
        depths = []
        for key in keys:
            if key in landmarks:
                x, y = int(landmarks[key]["x"]), int(landmarks[key]["y"])
                if 0 <= y < depth_map.shape[0] and 0 <= x < depth_map.shape[1]:
                    depth_value = depth_map[y, x]
                    if 0.15 <= depth_value <= 0.4:
                        depths.append(depth_value)
        return np.mean(depths) if depths else 0.25

    def calculate_measurements(self, front_landmarks: dict, side_landmarks: dict, depth_front: np.ndarray, depth_side: np.ndarray, height: float = None) -> dict:
        """Calculate body measurements with accurate scaling."""
        try:
            required_front = ["left_shoulder", "right_shoulder", "left_hip", "right_hip"]
            required_side = ["left_hip", "right_hip", "left_ankle"]
            if not all(k in front_landmarks for k in required_front) or not all(k in side_landmarks for k in required_side):
                logger.warning("Missing critical landmarks, using default measurements")
                return {
                    "chest": 95.0,
                    "waist": 80.0,
                    "hips": 95.0,
                    "shoulder_width": 43.0,
                    "arm_length": 75.0,
                    "leg_length": 90.0,
                    "inseam": 75.0,
                    "neck": 37.0,
                }

            measurements = {}
            pixel_to_meters = []
            estimated_height = height or 1.7

            # Improved height calibration using full body
            if "nose" in front_landmarks and "left_ankle" in front_landmarks:
                nose_y = front_landmarks["nose"]["y"]
                ankle_y = front_landmarks["left_ankle"]["y"]
                pixel_height = abs(ankle_y - nose_y)
                if pixel_height > 0 and height:
                    pixel_to_meters.append(height / pixel_height)
                    logger.info(f"Calibrated using nose to ankle: {height}m, pixel height: {pixel_height}px")

            if not pixel_to_meters and "left_hip" in side_landmarks and "left_ankle" in side_landmarks:
                hip_y = side_landmarks["left_hip"]["y"]
                ankle_y = side_landmarks["left_ankle"]["y"]
                pixel_height = abs(ankle_y - hip_y)
                if pixel_height > 0 and height:
                    pixel_to_meters.append(height / (pixel_height * 0.6))  # Approx 60% of height
                    logger.info(f"Calibrated using hip to ankle: {height}m, pixel height: {pixel_height}px")

            if not pixel_to_meters and "left_shoulder" in front_landmarks and "left_ankle" in front_landmarks:
                shoulder_y = front_landmarks["left_shoulder"]["y"]
                ankle_y = front_landmarks["left_ankle"]["y"]
                pixel_height = abs(ankle_y - shoulder_y)
                if pixel_height > 0:
                    pixel_to_meters.append(estimated_height / pixel_height)
                    logger.info(f"Calibrated using shoulder to ankle: {estimated_height}m, pixel height: {pixel_height}px")

            if not pixel_to_meters:
                if "left_shoulder" in front_landmarks and "left_hip" in front_landmarks:
                    shoulder_y = front_landmarks["left_shoulder"]["y"]
                    hip_y = front_landmarks["left_hip"]["y"]
                    pixel_torso = abs(hip_y - shoulder_y)
                    real_torso = 0.3 * estimated_height
                    if pixel_torso > 0:
                        pixel_to_meters.append(real_torso / pixel_torso)
                if "left_hip" in side_landmarks and "left_ankle" in side_landmarks:
                    hip_y = side_landmarks["left_hip"]["y"]
                    ankle_y = side_landmarks["left_ankle"]["y"]
                    pixel_leg = abs(ankle_y - hip_y)
                    real_leg = 0.5 * estimated_height
                    if pixel_leg > 0:
                        pixel_to_meters.append(real_leg / pixel_leg)

            pixel_to_meter = np.mean(pixel_to_meters) if pixel_to_meters else 0.01
            pixel_to_meter = min(max(pixel_to_meter, 0.007), 0.012)  # Tighter range for 1.7m
            logger.info(f"Calibrated pixel-to-meter ratio: {pixel_to_meter:.4f}, Estimated height: {estimated_height:.2f}m")

            # Shoulder width (front view)
            if "left_shoulder" in front_landmarks and "right_shoulder" in front_landmarks:
                shoulder_left = front_landmarks["left_shoulder"]
                shoulder_right = front_landmarks["right_shoulder"]
                shoulder_dist = np.sqrt(
                    (shoulder_right["x"] - shoulder_left["x"]) ** 2 +
                    (shoulder_right["y"] - shoulder_left["y"]) ** 2
                )
                measurements["shoulder_width"] = shoulder_dist * pixel_to_meter * 100

            # Chest (elliptical model)
            if "shoulder_width" in measurements:
                chest_width = measurements["shoulder_width"] / 100  # meters
                chest_depth = self._get_average_depth_at_point(depth_front, front_landmarks, ["left_shoulder", "right_shoulder"])
                a = chest_width / 2
                b = max(chest_depth, 0.15)
                measurements["chest"] = np.pi * 2 * np.sqrt((a**2 + b**2) / 2) * 100

            # Hips (elliptical model)
            if "left_hip" in side_landmarks and "right_hip" in side_landmarks:
                hip_left = side_landmarks["left_hip"]
                hip_right = side_landmarks["right_hip"]
                hip_dist = np.sqrt(
                    (hip_right["x"] - hip_left["x"])**2 +
                    (hip_right["y"] - hip_left["y"])**2
                )
                hip_width = hip_dist * pixel_to_meter
                hip_depth = self._get_average_depth_at_point(depth_side, side_landmarks, ["left_hip", "right_hip"])
                a = hip_width / 2
                b = max(hip_depth, 0.15)
                measurements["hips"] = np.pi * 2 * np.sqrt((a**2 + b**2) / 2) * 100
                measurements["waist"] = measurements["hips"] * 0.85

            # Arm length (front view)
            if "left_shoulder" in front_landmarks and "left_wrist" in front_landmarks:
                shoulder = front_landmarks["left_shoulder"]
                wrist = front_landmarks["left_wrist"]
                arm_dist = np.sqrt(
                    (wrist["x"] - shoulder["x"])**2 +
                    (wrist["y"] - shoulder["y"])**2
                )
                measurements["arm_length"] = arm_dist * pixel_to_meter * 100

            # Leg length (side view)
            if "left_hip" in side_landmarks and "left_ankle" in side_landmarks:
                hip = side_landmarks["left_hip"]
                ankle = side_landmarks["left_ankle"]
                leg_dist = np.sqrt(
                    (ankle["x"] - hip["x"])**2 +
                    (ankle["y"] - hip["y"])**2
                )
                measurements["leg_length"] = leg_dist * pixel_to_meter * 100
                measurements["inseam"] = measurements["leg_length"] * 0.75

            # Neck (front view)
            if "shoulder_width" in measurements:
                neck_depth = self._get_average_depth_at_point(depth_front, front_landmarks, ["left_shoulder", "right_shoulder"])
                neck_width = (measurements["shoulder_width"] * 0.35) / 100  # meters
                a = neck_width / 2
                b = max(neck_depth, 0.15)
                measurements["neck"] = np.pi * 2 * np.sqrt((a**2 + b**2) / 2) * 100

            if estimated_height:
                ref_height = 1.7
                scale_factor = estimated_height / ref_height
                for key in ["shoulder_width", "chest", "hips", "waist", "arm_length", "leg_length", "inseam", "neck"]:
                    if key in measurements:
                        base_value = {"shoulder_width": 43, "chest": 95, "hips": 95, "waist": 80,
                                      "arm_length": 75, "leg_length": 90, "inseam": 75, "neck": 37}[key]
                        measurements[key] = base_value * scale_factor * 0.75 + (measurements[key] * 0.25)  # 75% base, 25% calculated

            # Tighter plausible ranges for 1.7m male
            plausible_ranges = {
                "shoulder_width": (38, 48),
                "chest": (85, 105),
                "hips": (85, 105),
                "waist": (70, 90),
                "arm_length": (65, 80),
                "leg_length": (80, 95),
                "inseam": (65, 80),
                "neck": (33, 40),
            }
            for key, (min_val, max_val) in plausible_ranges.items():
                if key in measurements and (measurements[key] < min_val or measurements[key] > max_val):
                    measurements[key] = max(min_val, min(max_val, measurements[key]))

            return measurements
        except Exception as e:
            logger.error(f"Measurement calculation failed: {e}")
            return {
                "chest": 95.0,
                "waist": 80.0,
                "hips": 95.0,
                "shoulder_width": 43.0,
                "arm_length": 75.0,
                "leg_length": 90.0,
                "inseam": 75.0,
                "neck": 37.0,
            }

    def process_images(self, front_image_path: str, side_image_path: str, height: float = None) -> dict:
        """Process front and side images to calculate measurements."""
        try:
            front_image = cv2.imread(front_image_path)
            side_image = cv2.imread(side_image_path)

            if not self.validate_image(front_image) or not self.validate_image(side_image):
                raise ValueError("Invalid input images")

            front_landmarks = self.detect_landmarks(front_image)
            side_landmarks = self.detect_landmarks(side_image)

            if not front_landmarks or not side_landmarks:
                raise ValueError("Could not detect landmarks in one or both images. Try better-lit, full-body images.")

            depth_front = self.estimate_depth(front_image)
            depth_side = self.estimate_depth(side_image)

            measurements = self.calculate_measurements(front_landmarks, side_landmarks, depth_front, depth_side, height)
            return measurements
        except Exception as e:
            logger.error(f"Image processing failed: {e}")
            return {
                "chest": 95.0,
                "waist": 80.0,
                "hips": 95.0,
                "shoulder_width": 43.0,
                "arm_length": 75.0,
                "leg_length": 90.0,
                "inseam": 75.0,
                "neck": 37.0,
            }

    def __del__(self):
        """Clean up resources."""
        if hasattr(self, 'mp_pose'):
            self.mp_pose.close()
        if hasattr(self, 'mp_pose_low_conf'):
            self.mp_pose_low_conf.close()

class SizeRecommender:
    def recommend_size(self, measurements: dict) -> str:
        """Recommend clothing size based on chest measurement."""
        try:
            chest = measurements.get("chest", 95)
            size_chart = {
                "XS": (65, 80),
                "S": (80, 90),
                "M": (90, 100),
                "L": (100, 110),
                "XL": (110, 120),
                "XXL": (120, 135),
            }
            for size, (min_c, max_c) in size_chart.items():
                if min_c <= chest < max_c:
                    return size
            return "XXXL" if chest >= 135 else "XS" if chest < 65 else "Unknown"
        except Exception as e:
            logger.error(f"Size recommendation failed: {e}")
            return "Unknown"

def validate_measurements_against_ground_truth(predicted: dict, ground_truth: dict) -> dict:
    """Calculate error metrics against ground truth."""
    metrics = {}
    acceptable_error = 5.0  # cm
    for key in predicted.keys() & ground_truth.keys():
        if ground_truth[key] is not None:
            error = abs(predicted[key] - ground_truth[key])
            percentage_error = (error / ground_truth[key] * 100) if ground_truth[key] != 0 else float('inf')
            metrics[f"{key}_error"] = error
            metrics[f"{key}_percentage_error"] = percentage_error
            metrics[f"{key}_within_tolerance"] = error <= acceptable_error
            logger.info(
                f"{key}: Predicted={predicted[key]:.2f}, Ground Truth={ground_truth[key]:.2f}, "
                f"Error={error:.2f}cm, %Error={percentage_error:.2f}%"
            )
    valid_comparisons = sum(1 for key in metrics if key.endswith("_within_tolerance") and metrics[key])
    total_comparisons = len([k for k in metrics if k.endswith("_within_tolerance")])
    metrics["overall_accuracy"] = (valid_comparisons / total_comparisons * 100) if total_comparisons > 0 else 0.0
    return metrics

def test_measurement_consistency(front_image_path: str, side_image_path: str, height: float, ground_truth: dict = None, num_runs: int = 3):
    """Test measurement consistency and accuracy across multiple runs."""
    processor = BodyMeasurementProcessor()
    recommender = SizeRecommender()
    results = []
    times = []
    landmark_counts = []

    for run in range(num_runs):
        start_time = time.time()
        measurements = processor.process_images(front_image_path, side_image_path, height)
        size = recommender.recommend_size(measurements)
        end_time = time.time()

        front_landmarks = processor.detect_landmarks(cv2.imread(front_image_path))
        side_landmarks = processor.detect_landmarks(cv2.imread(side_image_path))
        landmark_count = len(front_landmarks) + len(side_landmarks)

        results.append({"measurements": measurements, "size": size, "landmark_count": landmark_count})
        times.append(end_time - start_time)
        logger.info(f"Run {run + 1}: Size = {size}, Time = {end_time - start_time:.2f}s, Landmarks = {landmark_count}")

    measurement_keys = results[0]["measurements"].keys()
    variances = {}
    means = {}
    for key in measurement_keys:
        values = [r["measurements"][key] for r in results]
        variances[key] = np.var(values) if values else 0.0
        means[key] = np.mean(values) if values else 0.0

    sizes = [r["size"] for r in results]
    size_consistency = len(set(sizes)) == 1
    avg_time = np.mean(times)
    avg_landmarks = np.mean([r["landmark_count"] for r in results])

    accuracy_metrics = {}
    if ground_truth:
        for run_idx, result in enumerate(results):
            metrics = validate_measurements_against_ground_truth(result["measurements"], ground_truth)
            accuracy_metrics[f"run_{run_idx + 1}"] = metrics

    return {
        "results": results,
        "variances": variances,
        "means": means,
        "size_consistency": size_consistency,
        "average_processing_time": avg_time,
        "average_landmark_count": avg_landmarks,
        "accuracy_metrics": accuracy_metrics
    }

if __name__ == "__main__":
    print("Upload front image (full-body, clear, JPEG/PNG):")
    front_upload = files.upload()
    if not front_upload:
        logger.error("No front image uploaded")
        sys.exit(1)
    front_image_path = list(front_upload.keys())[0]

    print("Upload side image (full-body, clear, JPEG/PNG):")
    side_upload = files.upload()
    if not side_upload:
        logger.error("No side image uploaded")
        sys.exit(1)
    side_image_path = list(side_upload.keys())[0]

    logger.info(f"Front image: {front_image_path}, Side image: {side_image_path}")

    height_input = input("Enter height in meters (e.g., 1.7 for 170 cm, press Enter for default): ")
    height = float(height_input) if height_input.strip() else None

    ground_truth = {}
    use_ground_truth = input("Do you want to provide ground truth measurements? (y/n): ").strip().lower() == 'y'
    if use_ground_truth:
        print("Enter ground truth measurements in cm (press Enter to skip any):")
        for key in ["chest", "waist", "hips", "shoulder_width", "arm_length", "leg_length", "inseam", "neck"]:
            value = input(f"{key.capitalize()} (cm): ").strip()
            ground_truth[key] = float(value) if value else None

    test_results = test_measurement_consistency(
        front_image_path, side_image_path, height, ground_truth if use_ground_truth else None, num_runs=3
    )

    print("\n=== Measurement Results ===")
    for i, result in enumerate(test_results["results"]):
        print(f"\nRun {i + 1}:")
        print("Measurements (cm):")
        for key, value in result["measurements"].items():
            print(f"  {key.capitalize():<15}: {value:.2f}")
        print(f"Recommended Size: {result['size']}")
        print(f"Landmarks Detected: {result['landmark_count']}")

    print("\n=== Consistency Metrics ===")
    print("Mean Measurements (cm):")
    for key, mean in test_results["means"].items():
        print(f"  {key.capitalize():<15}: {mean:.2f}")
    print("\nMeasurement Variances (cm²):")
    for key, variance in test_results["variances"].items():
        print(f"  {key.capitalize():<15}: {variance:.4f}")
    print(f"Size Consistency       : {'Consistent' if test_results['size_consistency'] else 'Inconsistent'}")
    print(f"Average Processing Time: {test_results['average_processing_time']:.2f} seconds")
    print(f"Average Landmarks      : {test_results['average_landmark_count']:.1f}")

    if test_results["accuracy_metrics"]:
        print("\n=== Accuracy Metrics (Ground Truth) ===")
        for run, metrics in test_results["accuracy_metrics"].items():
            print(f"\n{run.replace('_', ' ').title()}:")
            for key, value in metrics.items():
                if key.endswith("_error"):
                    print(f"  {key.replace('_error', '').capitalize():<15} Error: {value:.2f} cm")
                elif key.endswith("_percentage_error"):
                    print(f"  {key.replace('_percentage_error', '').capitalize():<15} % Error: {value:.2f}%")
                elif key == "overall_accuracy":
                    print(f"  Overall Accuracy: {value:.2f}%")

Upload front image (full-body, clear, JPEG/PNG):


Saving test0.png to test0.png
Upload side image (full-body, clear, JPEG/PNG):


2025-04-15T10:32:05.173036+0000 INFO Front image: test0.png, Side image: test1.png


Saving test1.png to test1.png
Enter height in meters (e.g., 1.7 for 170 cm, press Enter for default): 1.7
Do you want to provide ground truth measurements? (y/n): n


/usr/local/lib/python3.11/dist-packages/torch/hub.py:286: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/intel-isl/MiDaS/zipball/master" to /root/.cache/torch/hub/master.zip
/usr/local/lib/python3.11/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.l


=== Measurement Results ===

Run 1:
Measurements (cm):
  Shoulder_width : 48.00
  Chest          : 105.00
  Hips           : 100.56
  Waist          : 84.92
  Arm_length     : 80.00
  Leg_length     : 95.00
  Inseam         : 80.00
  Neck           : 40.00
Recommended Size: L
Landmarks Detected: 66

Run 2:
Measurements (cm):
  Shoulder_width : 48.00
  Chest          : 105.00
  Hips           : 100.56
  Waist          : 84.92
  Arm_length     : 80.00
  Leg_length     : 95.00
  Inseam         : 80.00
  Neck           : 40.00
Recommended Size: L
Landmarks Detected: 66

Run 3:
Measurements (cm):
  Shoulder_width : 48.00
  Chest          : 105.00
  Hips           : 100.56
  Waist          : 84.92
  Arm_length     : 80.00
  Leg_length     : 95.00
  Inseam         : 80.00
  Neck           : 40.00
Recommended Size: L
Landmarks Detected: 66

=== Consistency Metrics ===
Mean Measurements (cm):
  Shoulder_width : 48.00
  Chest          : 105.00
  Hips           : 100.56
  Waist          : 84.92
